# Пятое домашнее задание
---

**Студент:** Феоктистов Владислав Сергеевич

**Группа:** НПМбд-01-19





## Предисловие

Перед реализацией алгоритмов LU,LUP-разложений необходимо сделать некоторые приготовления. Сперва необходимо импортировать необходимые библиотеки для дальнейшей работы и создать класс **LinEq** (от linear equations), где будут реализованы все функции.

**Код:**

In [12]:
import numpy as np
import scipy.linalg


class LinEq:
    """
    The class of static functions for LU, LUP decomposition of a matrix and
    finding a solution to a system of linear algebraic equations.
    """

После добавим функцию **__swap_strings(A, i, j)** для перестановки строк i и j; функцию **__check_matrix(A)** для проверки исходной матрицы на невырожденность, размерность и соответсии квадратичной форме; а также функцию **convert_p_to_scipy_p(P)** для конвертации матрицы P из нашего LUP-разложения к матрицы P из функции scipy.linalg.lu() из библиотеки scipy (т.е. матрица перестановок P переходит из вида PA = LU к виду A = PLU).

**Код:**

In [9]:
@staticmethod
def __check_matrix(A):
    """
    The function to check the correctness of the supplied matrix A for LU and LUP decomposition.
    The matrix must be non-degenerate, square and have a rank of at least two.
    Otherwise, an exception will be thrown.
    :param A: matrix for checking
    :return: None
    """
    if np.linalg.det(A) == 0:
        raise Exception('Invalid matrix! Degenerate matrix!')
    if A.shape[0] != A.shape[1]:
        raise Exception('Invalid matrix! Not a square matrix!')
    if A.shape[0] < 2:
        raise Exception('Invalid matrix! Low-dimension matrix!')

@staticmethod
def __swap_strings(A, i, j):
    """
    The function rearranges the two specified rows in the matrix.
    :param A: the original matrix
    :param i: first index for permutation
    :param j: second index for permutation
    :return: None
    """
    if not 0 <= i < A.shape[0] or not 0 <= j < A.shape[0]:
        raise Exception('The specified index is out of range!')
    if i != j:
        A[i], A[j] = A[j], np.copy(A[i])
            
@staticmethod
def convert_p_to_scipy_p(P):
    """
    The function for converting the permutation matrix P to the scipy
    library standard (A = PLU).
    :param P: the original P matrix (PA = LU)
    :return: converted P matrix to scipy standards (A = PLU)
    """
    return P.transpose()

## LU-разложение

**LU-разложение (LU-декомпозиция, LU-факторизация)** — представление матрицы A в виде произведения двух матриц, A = LU, где L — нижняя треугольная матрица, а U — верхняя треугольная матрица.

LU-разложение используется для решения систем линейных уравнений, обращения матриц и вычисления определителя. LU-разложение существует только в том случае, когда матрица A обратима, а все ведущие (угловые) главные миноры матрицы A невырождены.

Этот метод является одной из разновидностей метода Гаусса.


### Алгоритм нахождения матриц L и U

Создаем копию U матрицы A и единичную матрицу L. Матрицу U мы будем приводить к ступенчатому виду по алгоритму Гаусса, а в матрицу L записывать коэффициенты умножения i-ой строки при переходе к ступенчатому виду (из j-ой строки вычитается i-ая строка, умноженная на коэффициент равный A[j, i] / A[i, i]).

Коэффициенты для уможения i-ой строки (i-ый шаг преобразования, т.е. шаг обнуление строк под i-ой строкой в i-ом столбце) можно найти, поделив весь i-ый столбец под i-ой строкой на число A[i, i]. Таким образом, мы получили i-ый вектор коэффициентов преобразования, который необходимо подставить снизу вверх в i-ый столбец матрицы L.

Далее необходимо вычесть из j-ых строк (j = i+1,...,n) i-ую строку, умноженную на L[i, j] (найденный коэффициент преобразования). Это можно сделать, вычтив поэлементно из квадранта, находящегося под i-ой строкой, квадрант равный поэлементному уможению i-го вектора преобразования на i-ую строку.

**Код:**

In [8]:
@staticmethod
def lu_decomposition(A):
    """
    The function for LU decomposition, representing the matrix A as a product of two matrices,
    A = LU, where L is the lower triangular matrix and U is the upper triangular matrix.
    :param A: matrix for decomposition
    :return: L and U matrices
    """
    LinEq.__check_matrix(A)
    n = A.shape[0]
    U = np.copy(A)
    L = np.identity(n)
    for i in range(n - 1):
        L[i + 1:, i] = U[:, i][i + 1:] / U[i, i]
        U[i + 1:, ] -= L[i + 1:, i].reshape((n - i - 1, 1)) * U[i]
    return L, U

### Алгоритм решения системы уравнения методом LU-преобразования

Полученное LU-разложение матрицы A (матрица коэффициентов системы) может быть использовано для решения семейства систем линейных уравнений с различными векторами b в правой части:

\begin{align} Ax = b \end{align}

Если известно LU-разложение матрицы A, A = LU, исходная система может быть записана как

\begin{align} LUx = b \end{align}

Эта система может быть решена в два шага. На первом шаге решается система

\begin{align} Ly = b \end{align}

Поскольку L — нижняя треугольная матрица, эта система решается непосредственно прямой подстановкой. Нулевой элемент вектора y будет равен нулевому элементу вектора b, т.к. A[0, 0] = 1, т.е. y[0] = b[0]. Последующие n-1 строк вектора y будут получаться путем вычитания из i-го элемента вектора b скалярного произведения i-ой строки c элементами от 0 до i-1 вектора y с элементами от 0 до i-1, т.е. того же размера.

На втором шаге решается система

\begin{align} Ux = y \end{align}

Поскольку U — верхняя треугольная матрица, эта система решается непосредственно обратной подстановкой. Последний элемент вектора y будет равен последнему элементу вектора b, деленного на U[i, i], т.е. y[n - 1] = b[n - 1] / U[n - 1, n - 1]. Последующие n-1 строк вектора y будут получаться путем вычитания из i-го элемента вектора b скалярного произведения i-ой строки c элементами от i+1 до n-1 вектора y с элементами от i+1 до n-1, умноженного на 1 / U[i, i]. Вычисления здесь уже идут снизу вверх.

**Код:**

In [17]:
@staticmethod
def __forward_substitution(L, b):
    """
    The function for a system of linear algebraic equations with a lower
    triangular matrix of coefficients by a forward substitution
    :param L: lower triangular matrix
    :param b: vector of absolute terms
    :return: solution vector y
    """
    n = b.shape[0]
    y = np.zeros(n, dtype=float)
    y[0] = b[0]
    for i in range(1, n):
        y[i] = b[i] - (L[i][:i] @ y[:i])
    return y

@staticmethod
def __back_substitution(U, y):
    """
    The function for a system of linear algebraic equations with an upper
    triangular matrix of coefficients by a back substitution
    :param U: upper triangular matrix
    :param y: vector of absolute terms
    :return: solution vector x
    """
    n = y.shape[0]
    x = np.zeros(n, dtype=float)
    x[n - 1] = y[n - 1] / U[n - 1][n - 1]
    for i in range(n - 2, -1, -1):
        x[i] = (y[i] - (U[i][i + 1:] @ x[i + 1:])) / U[i, i]
    return x

@staticmethod
def __lu_solver(L, U, b):
    """
    The function for solving a system of linear algebraic
    equations by the LU decomposition method.
    :param L: lower triangular matrix
    :param U: upper triangular matrix
    :param b: vector of absolute terms
    :return: solution vector
    """
    y = LinEq.__forward_substitution(L, b)
    x = LinEq.__back_substitution(U, y)
    return x

@staticmethod
def lu_solver(A, b):
    """
    The function for solving a system of linear algebraic
    equations by the LU decomposition method.
    :param A: matrix of coefficients of the system
    :param b: vector of absolute terms
    :return: solution vector
    """
    L, U = LinEq.lu_decomposition(A)
    return LinEq.__lu_solver(L, U, b)

Приватная функция **__forward_substitution(L, b)** решает СЛАУ с нижней треугольной матрицей коэффициентов путем прямой подстановки, т.е. эта функция подходит для решения системы Ly = b.

Приватная функция **__back_substitution(U, y)** решает СЛАУ с верхней треугольной матрицей коэффициентов путем обратной подстановки, т.е. эта функция подходит для решения системы Ux = y.

Приватная функция **__lu_solver(L, U, b)** реализует алгоритм нахождения решения СЛАУ (системы линейных алгебраических уравнений) методом LU-разложения. В качестве параметров передаются уже найденные матрицы L и U, а также вектор b из системы. Эта функция в дальнейшем упростит алгорит решения СЛАУ методом LUP-разложения.

Функция **lu_solver(A, b)** упрощает предыдущую функцию. В качестве параметров передаются только матрица A и вектор b.

## LUP-разложение

**LUP-разложение (LUP-декомпозиция)** — представление данной матрицы A в виде произведения PA = LU где матрица L является нижнетреугольной с единицами на главной диагонали, U — верхнетреугольная общего вида, а P — т. н. матрица перестановок, получаемая из единичной матрицы путём перестановки строк или столбцов. Такое разложение можно осуществить для любой невырожденной матрицы. LUP-разложение используется для вычисления обратной матрицы по компактной схеме, вычисления решения системы линейных уравнений. По сравнению с алгоритмом LU-разложения алгоритм LUP-разложения может обрабатывать любые невырожденные матрицы и при этом обладает более высокой вычислительной устойчивостью.

### Метод нахождения матриц L, U и P

Введем единичную матрицу P. Матрицы L и U находятся аналогичным образом, за исключением того, что вместе матрицей перестановок P будут перестанавливаться строки в матрицах L и U по следующему правилу: на каждом i-м шаге сначала производится поиск опорного (ведущего) элемента — максимального по модулю элемента среди элементов i-го столбца, находящихся не выше i-й строки, после чего строка с опорным элементом меняется местами с i-й строкой. Если матрица невырождена, то опорный элемент гарантированно будет отличен от нуля.

In [7]:
@staticmethod
def lup_decomposition(A):
    """
    The function for LUP decomposition, representing the matrix A as a product of PA = LU
    where the matrix L is lower triangular with units on the main diagonal, U is upper
    triangular in general form, and P is a permutation matrix obtained from the unit
    matrix by rearranging rows or columns.
    :param A: matrix for decomposition
    :return: P, L and U matrices
    """
    LinEq.__check_matrix(A)
    n = A.shape[0]
    U = np.copy(A)
    L = np.zeros((n, n))
    P = np.identity(n)
    for i in range(n - 1):
        col = U[:, i][i:]
        pivot = np.argmax(np.abs(col)) + i  # The reference element of the i-th column
        if pivot != i:
            LinEq.__swap_strings(U, i, pivot)
            LinEq.__swap_strings(L, i, pivot)
            LinEq.__swap_strings(P, i, pivot)
        L[i + 1:, i] = U[:, i][i + 1:] / U[i, i]
        U[i + 1:, ] -= L[i + 1:, i].reshape((n - i - 1, 1)) * U[i]
    return P, L + np.identity(n), U

### Алгоритм решения системы уравнения методом LUP-преобразования

Полученное LUP-разложение матрицы A может быть использовано для решения СЛАУ с различными векторами b в правой части:

\begin{align} Ax = b \end{align}

Если известно LUP-разложение матрицы A, A = P^(-1)*LU, исходная система может быть записана как

\begin{align} P^{-1}LUx = b \end{align} или \begin{align} LUx = Pb \end{align}

Эта система может быть решена в два шага. На первом шаге решается система

\begin{align} Ly = Pb \end{align}

Поскольку L — нижняя треугольная матрица, эта система решается непосредственно прямой подстановкой.

На втором шаге решается система

\begin{align} Ux = y \end{align}

Поскольку U — верхняя треугольная матрица, эта система решается непосредственно обратной подстановкой.

Как можно видеть, решение получается таким же, как и при решение СЛАУ методом LU-разложения, за исключением того, что нужно будет заменить b на Pb. Таким образом, для реализации функции нахождения решения СЛАУ методом LUP-разложения необходимо будет только использовать готовую функцию **__lu_solver(L, U, b)** c заменой b на Pb.

**Код:**

In [6]:
@staticmethod
def lup_solver(A, b):
    """
    The function for solving a system of linear algebraic
    equations by the LUP decomposition method.
    :param A: matrix of coefficients of the system
    :param b: vector of absolute terms
    :return: solution vector
    """
    P, L, U = LinEq.lup_decomposition(A)
    return LinEq.__lu_solver(L, U, P @ b)

## Финальный вид класса LinEq

In [10]:
import numpy as np
import scipy.linalg


class LinEq:
    """
    The class of static functions for LU, LUP decomposition of a matrix and
    finding a solution to a system of linear algebraic equations.
    """

    @staticmethod
    def __check_matrix(A):
        """
        The function to check the correctness of the supplied matrix A for LU and LUP decomposition.
        The matrix must be non-degenerate, square and have a rank of at least two.
        Otherwise, an exception will be thrown.
        :param A: matrix for checking
        :return: None
        """
        if np.linalg.det(A) == 0:
            raise Exception('Invalid matrix! Degenerate matrix!')
        if A.shape[0] != A.shape[1]:
            raise Exception('Invalid matrix! Not a square matrix!')
        if A.shape[0] < 2:
            raise Exception('Invalid matrix! Low-dimension matrix!')

    @staticmethod
    def __swap_strings(A, i, j):
        """
        The function rearranges the two specified rows in the matrix.
        :param A: the original matrix
        :param i: first index for permutation
        :param j: second index for permutation
        :return: None
        """
        if not 0 <= i < A.shape[0] or not 0 <= j < A.shape[0]:
            raise Exception('The specified index is out of range!')
        if i != j:
            A[i], A[j] = A[j], np.copy(A[i])
            
    @staticmethod
    def convert_p_to_scipy_p(P):
        """
        The function for converting the permutation matrix P to the scipy
        library standard (A = PLU).
        :param P: the original P matrix (PA = LU)
        :return: converted P matrix to scipy standards (A = PLU)
        """
        return P.transpose()

    @staticmethod
    def lu_decomposition(A):
        """
        The function for LU decomposition, representing the matrix A as a product of two matrices,
        A = LU, where L is the lower triangular matrix and U is the upper triangular matrix.
        :param A: matrix for decomposition
        :return: L and U matrices
        """
        LinEq.__check_matrix(A)
        n = A.shape[0]
        U = np.copy(A)
        L = np.identity(n)
        for i in range(n - 1):
            L[i + 1:, i] = U[:, i][i + 1:] / U[i, i]
            U[i + 1:, ] -= L[i + 1:, i].reshape((n - i - 1, 1)) * U[i]
        return L, U

    @staticmethod
    def lup_decomposition(A):
        """
        The function for LUP decomposition, representing the matrix A as a product of PA = LU
        where the matrix L is lower triangular with units on the main diagonal, U is upper
        triangular in general form, and P is a permutation matrix obtained from the unit
        matrix by rearranging rows or columns.
        :param A: matrix for decomposition
        :return: P, L and U matrices
        """
        LinEq.__check_matrix(A)
        n = A.shape[0]
        U = np.copy(A)
        L = np.zeros((n, n))
        P = np.identity(n)
        for i in range(n - 1):
            col = U[:, i][i:]
            pivot = np.argmax(np.abs(col)) + i  # The reference element of the i-th column
            if pivot != i:
                LinEq.__swap_strings(U, i, pivot)
                LinEq.__swap_strings(L, i, pivot)
                LinEq.__swap_strings(P, i, pivot)
            L[i + 1:, i] = U[:, i][i + 1:] / U[i, i]
            U[i + 1:, ] -= L[i + 1:, i].reshape((n - i - 1, 1)) * U[i]
        return P, L + np.identity(n), U

    @staticmethod
    def __forward_substitution(L, b):
        """
        The function for a system of linear algebraic equations with a lower
        triangular matrix of coefficients by a forward substitution
        :param L: lower triangular matrix
        :param b: vector of absolute terms
        :return: solution vector y
        """
        n = b.shape[0]
        y = np.zeros(n, dtype=float)
        y[0] = b[0]
        for i in range(1, n):
            y[i] = b[i] - (L[i][:i] @ y[:i])
        return y

    @staticmethod
    def __back_substitution(U, y):
        """
        The function for a system of linear algebraic equations with an upper
        triangular matrix of coefficients by a back substitution
        :param U: upper triangular matrix
        :param y: vector of absolute terms
        :return: solution vector x
        """
        n = y.shape[0]
        x = np.zeros(n, dtype=float)
        x[n - 1] = y[n - 1] / U[n - 1][n - 1]
        for i in range(n - 2, -1, -1):
            x[i] = (y[i] - (U[i][i + 1:] @ x[i + 1:])) / U[i, i]
        return x

    @staticmethod
    def __lu_solver(L, U, b):
        """
        The function for solving a system of linear algebraic
        equations by the LU decomposition method.
        :param L: lower triangular matrix
        :param U: upper triangular matrix
        :param b: vector of absolute terms
        :return: solution vector
        """
        y = LinEq.__forward_substitution(L, b)
        x = LinEq.__back_substitution(U, y)
        return x

    @staticmethod
    def lu_solver(A, b):
        """
        The function for solving a system of linear algebraic
        equations by the LU decomposition method.
        :param A: matrix of coefficients of the system
        :param b: vector of absolute terms
        :return: solution vector
        """
        L, U = LinEq.lu_decomposition(A)
        return LinEq.__lu_solver(L, U, b)

    @staticmethod
    def lup_solver(A, b):
        """
        The function for solving a system of linear algebraic
        equations by the LUP decomposition method.
        :param A: matrix of coefficients of the system
        :param b: vector of absolute terms
        :return: solution vector
        """
        P, L, U = LinEq.lup_decomposition(A)
        return LinEq.__lu_solver(L, U, P @ b)

## Проверка кода

Для проверки можно взять любую невырожденную матрицу A и любой вектор b. Для проверки используются результаты работы функции **scipy.linalg.lu()** из библиотеки scipy. Наша матрица P конвертируется под стандарты используемой библиотеки, а некоторые переменные окргляются до 8го знака после запятой, чтобы избежать компьютерные ошибки при сравненнии и вычислении.

Тесты реализованы с помощью инструкции assert с пояснениями при возникновении ошибки. В конце также выводятся результаты вычислений.

**Код:**

In [11]:
if __name__ == "__main__":
    A = np.array([
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 10]
    ], dtype=float)
    b = np.array([1, 0, 0], dtype=float)

    L1, U1 = LinEq.lu_decomposition(A)  # Matrices L,U from the LU decomposition
    P2, L2, U2 = LinEq.lup_decomposition(A)  # Matrices L,U from the LUP decomposition
    P3, L3, U3 = scipy.linalg.lu(A)  # Matrices L,U from the LU decomposition from scipy library
    conv_P = LinEq.convert_p_to_scipy_p(P2)  # Converted matrix P2 to scipy standards

    my_lu_A = L1 @ U1  # Obtaining the original matrix A after LU decomposition. A = LU
    my_plu_A = conv_P @ L2 @ U2  # Obtaining the original matrix A after LUP decomposition. PA = LU
    scipy_A = P3 @ L3 @ U3  # Obtaining the original matrix A after LU decomposition using the scipy library. A = PLU

    X1 = LinEq.lu_solver(A, b)  # Solving a system of linear algebraic equations by the LU decomposition method
    X2 = LinEq.lup_solver(A, b)  # Solving a system of linear algebraic equations by the LUP decomposition method
    X3 = scipy.linalg.solve(A, b)  # Solving a system of linear algebraic equations using the scipy library

    # Rounding to the 8th decimal place to avoid incorrect comparison

    L2 = np.round(L2, 8)
    U2 = np.round(U2, 8)

    L3 = np.round(L3, 8)
    U3 = np.round(U3, 8)

    X1 = np.round(X1, 8)
    X2 = np.round(X2, 8)
    X3 = np.round(X3, 8)

    # Check all solutions

    assert np.all(conv_P == P3), '(Converted P2) not equals P3'
    assert np.all(L2 == L3), 'L2 not equals L3'
    assert np.all(U2 == U3), 'U2 not equals U3'

    assert np.all(A == my_lu_A), 'A not equals my_lu_A'
    assert np.all(A == my_plu_A), 'A not equals my_plu_A'
    assert np.all(A == scipy_A), 'A not equals scipy_A'

    assert np.all(X1 == X3), \
        'Incorrect solution of a system of linear algebraic equations by the LU decomposition method'
    assert np.all(X2 == X3), \
        'Incorrect solution of a system of linear algebraic equations by the LUP decomposition method'

    print('All tests were successfully passed')

    # Output of results

    print('\nL matrix from LU decomposition method: \n', L1)
    print('\nU matrix from LU decomposition method: \n', U1)

    print('\nP matrix from LUP decomposition method: \n', P2)
    print('\nL matrix from LUP decomposition method: \n', L2)
    print('\nU matrix from LUP decomposition method: \n', U2)

    print('\nLU solution:\n', X1)
    print('\nLUP solution:\n', X2)

All tests were successfully passed

L matrix from LU decomposition method: 
 [[1. 0. 0.]
 [4. 1. 0.]
 [7. 2. 1.]]

U matrix from LU decomposition method: 
 [[ 1.  2.  3.]
 [ 0. -3. -6.]
 [ 0.  0.  1.]]

P matrix from LUP decomposition method: 
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

L matrix from LUP decomposition method: 
 [[1.         0.         0.        ]
 [0.14285714 1.         0.        ]
 [0.57142857 0.5        1.        ]]

U matrix from LUP decomposition method: 
 [[ 7.          8.         10.        ]
 [ 0.          0.85714286  1.57142857]
 [ 0.          0.         -0.5       ]]

LU solution:
 [-0.66666667 -0.66666667  1.        ]

LUP solution:
 [-0.66666667 -0.66666667  1.        ]


## Итоги

Как Вы можете убедиться на примере, все тесты были успешно пройдены.

Во избежании деления на ноль при решении СЛАУ предпочтительнее выбирать LUP-разложение. В случае невырожденности матрицы A, гарантируется исключение возникновения этого рода ошибки. Чтобы убедиться в этом достаточно поменять в приведенном примере значение в первой строке первом столбце на 0. Важно отметить, что ошибка деления на ноль при использовании LU-разложения может возникнуть в процессе работы, когда при переходе матрицы к ступенчатому виду на главной диагонали появится ноль.